In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
currency_pair = 'Eur_Usd'
rounding = 3 if 'Jpy' in currency_pair else 5
pips_multiplier = 100 if 'Jpy' in currency_pair else 10000
year_range = '2022-2023'
fractal_window = 20
divider = 100 if 'Jpy' in currency_pair else 10000

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{year_range}.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_long = pd.read_csv(file_path + f'Oanda_{currency_pair}_H1_{year_range}.csv')
df_long.Date = pd.to_datetime(df_long.Date)
df_long.reset_index(drop=True, inplace=True)

In [5]:
def squeeze(barsdata, length=20, length_kc=20, mult=2.0):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    upper_kc = m_avg + range_ma * mult
    lower_kc = m_avg - range_ma * mult

    # Squeeze
    squeeze_on = (lower_bb > lower_kc) & (upper_bb < upper_kc)

    return squeeze_on

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

def fractal(lows, highs, atr_lb, atr_ub, window=20):
    assert len(lows) == len(highs)

    fractal_period = 2 * window + 1

    is_support = lows.rolling(fractal_period, center=True).apply(lambda x: x[window] == min(x), raw=True)
    is_resistance = highs.rolling(fractal_period, center=True).apply(lambda x: x[window] == max(x), raw=True)
    
    is_support_indices = pd.Series(is_support.index[is_support == 1.0])
    is_resistance_indices = pd.Series(is_resistance.index[is_resistance == 1.0])

    support_fractal_vals = lows[is_support_indices].reindex(lows.index).ffill()
    resistance_fractal_vals = highs[is_resistance_indices].reindex(highs.index).ffill()

    atr_lb_fractal_vals = atr_lb[is_support_indices].reindex(atr_lb.index).ffill()
    atr_ub_fractal_vals = atr_ub[is_resistance_indices].reindex(atr_ub.index).ffill()

    return support_fractal_vals, resistance_fractal_vals, atr_lb_fractal_vals, atr_ub_fractal_vals

def choc(closes, support_fractals, resistance_fractals):
    broke_down = closes < support_fractals
    broke_up = closes > resistance_fractals

    assert len(broke_down) == len(broke_up)
    
    choc, prev_val = [], None

    for i in range(len(broke_down)):
        # First occurrence
        if prev_val is None:
            if broke_down[i]:
                prev_val = 'broke_down'
                choc.append('broke_down')

            elif broke_up[i]:
                prev_val = 'broke_up'
                choc.append('broke_up')

            else:
                choc.append('na')

        # All other occurences
        elif broke_down[i] and prev_val == 'broke_up':
            prev_val = 'broke_down'
            choc.append('broke_down')

        elif broke_up[i] and prev_val == 'broke_down':
            prev_val = 'broke_up'
            choc.append('broke_up')

        else:
            choc.append('na')

    return pd.Series(choc)

In [6]:
df['lower_atr_band'], df['upper_atr_band'] = atr_bands(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['atr'] = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'])
df['squeeze'] = squeeze(df)
df['support_fractal'], df['resistance_fractal'], df['atr_lb_fractal'], df['atr_ub_fractal'] = fractal(df['Mid_Low'], df['Mid_High'], df['lower_atr_band'], df['upper_atr_band'], fractal_window)
df['support_fractal'], df['resistance_fractal'], df['atr_lb_fractal'], df['atr_ub_fractal'] = df['support_fractal'].shift(fractal_window), df['resistance_fractal'].shift(fractal_window), df['atr_lb_fractal'].shift(fractal_window), df['atr_ub_fractal'].shift(fractal_window)
df['choc'] = choc(df['Mid_Close'], df['support_fractal'], df['resistance_fractal'])
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

df_long['support_fractal'], df_long['resistance_fractal'], _, _ = fractal(df_long['Mid_Low'], df_long['Mid_High'], df['lower_atr_band'], df['upper_atr_band'], fractal_window)
df_long['support_fractal'], df_long['resistance_fractal'] = df_long['support_fractal'].shift(fractal_window), df_long['resistance_fractal'].shift(fractal_window)
df_long['choc'] = choc(df_long['Mid_Close'], df_long['support_fractal'], df_long['resistance_fractal'])
df_long.dropna(inplace=True)
df_long.reset_index(drop=True, inplace=True)

In [7]:
# df_long[['Date', 'Mid_Low', 'support_fractal', 'Mid_High', 'resistance_fractal', 'Mid_Close', 'choc']].iloc[-150:-100, :]

In [10]:
value_per_pip = 1.0
amounts_per_day = [-0.008, -0.01, -0.012] if 'Jpy' in currency_pair else [-0.00008, -0.0001, -0.00012]

In [14]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def run_simulation(spread_cutoff, sl_method, choc_pips_diff_threshold, use_squeeze, use_trailing_sl, risk_reward_ratio):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked, win_amounts, loss_amounts, trade_dates = [], [], [], []
    day_fees = 0
    trade = None
    # choc_pips_diff_threshold /= divider
    prev_choc, prev_level = 'na', None

    for i in range(1, len(df)):
        curr_date = df.loc[df.index[i], 'Date']

        # ---------------------------------------------------
        # Get the most recent choc from the longer time frame
        # ---------------------------------------------------
        curr_long = df_long.loc[df_long['Date'] <= curr_date]

        if len(curr_long) < 2:
            continue

        if len(df_long.loc[df_long.Date >= curr_date]) == 0:
            break

        prev_long_chocs = df_long.loc[(df_long['Date'] <= curr_long.loc[curr_long.index[-2], 'Date']) & (df_long['choc'] != 'na')]

        if len(prev_long_chocs) == 0:
            continue

        prev_choc = prev_long_chocs.loc[prev_long_chocs.index[-1], 'choc']

        assert prev_choc == 'broke_down' or prev_choc == 'broke_up'

        if prev_choc == 'broke_down':
            prev_level = prev_long_chocs.loc[prev_long_chocs.index[-1], 'support_fractal']

        else:
            prev_level = prev_long_chocs.loc[prev_long_chocs.index[-1], 'resistance_fractal']

        # print(curr_long.loc[curr_long.index[-2], 'Date'], prev_choc, prev_level)
        
        # ---------------------------------------------------
        # ---------------------------------------------------
        # ---------------------------------------------------

        curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high, curr_bid_low, curr_ask_high, curr_bid_close, curr_ask_close = df.loc[df.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Ask_High', 'Bid_Close', 'Ask_Close']]
        spread = abs(curr_ao - curr_bo)

        if trade is None:
            choc, support_fractal, resistance_fractal, atr_lb, atr_ub, atr_fractal_lb, atr_fractal_ub, squeeze_on, atr1 = df.loc[df.index[i - 1], ['choc', 'support_fractal', 'resistance_fractal', 'lower_atr_band', 'upper_atr_band', 'atr_lb_fractal', 'atr_ub_fractal', 'squeeze', 'atr']]
            curr_level = support_fractal if prev_choc == 'broke_down' else resistance_fractal
            curr_diff = abs(prev_level - curr_level)
            # within_distance_threshold = curr_diff <= choc_pips_diff_threshold
            within_distance_threshold = curr_diff <= (atr1 * choc_pips_diff_threshold)
            both_broke_up = choc == 'broke_up' and prev_choc == 'broke_up'
            both_broke_down = choc == 'broke_down' and prev_choc == 'broke_down'
            squeezed = squeeze_on if use_squeeze else True

            if both_broke_up and within_distance_threshold and squeezed:
                open_price = float(curr_ao)
                pullback = float(atr_fractal_lb) if sl_method == 'fractal_atr_band' else (float(support_fractal) if sl_method == 'fractal' else float(atr_lb))
                stop_loss = round(pullback, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), rounding)
                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_buys += 1
                        trade_dates.append(curr_date)

            elif both_broke_down and within_distance_threshold and squeezed:
                open_price = float(curr_bo)
                pullback = float(atr_fractal_ub) if sl_method == 'fractal_atr_band' else (float(resistance_fractal) if sl_method == 'fractal' else float(atr_ub))
                stop_loss = round(pullback, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), rounding)
                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                 'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5), 'n_units': n_units, 
                                 'original_units': n_units, 'start_date': curr_date, 'end_date': None, 'prev_profit_ratio': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_sells += 1
                        trade_dates.append(curr_date)

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and use_trailing_sl and curr_bid_close > trade['open_price']:
            curr_profit_ratio = (curr_bid_close - trade['open_price']) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] + (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'buy' and not use_trailing_sl and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and use_trailing_sl and curr_ask_close < trade['open_price']:
            curr_profit_ratio = (trade['open_price'] - curr_ask_close) / trade['pips_risked']

            # Initial move
            if curr_profit_ratio >= 1.0 and trade['prev_profit_ratio'] is None:
                trade['stop_loss'] = trade['open_price']
                trade['prev_profit_ratio'] = 0.0

            # if curr_profit_ratio >= 1.5 and trade['prev_profit_ratio'] == 0.0:
            #     trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * 0.5)
            #     trade['prev_profit_ratio'] = 0.5

            # Subsequent moves
            if curr_profit_ratio >= 2.0:
                # while curr_profit_ratio >= trade['prev_profit_ratio'] + 1.5:
                while curr_profit_ratio >= trade['prev_profit_ratio'] + 2.0:
                    # trade['prev_profit_ratio'] += 0.5
                    trade['prev_profit_ratio'] += 1.0
                    trade['stop_loss'] = trade['open_price'] - (trade['pips_risked'] * trade['prev_profit_ratio'])

        if trade is not None and trade['trade_type'] == 'sell' and not use_trailing_sl and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees = calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

    return reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts, trade_dates

In [15]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
spread_cutoffs = [0.1, 0.2]
# sl_methods = ['fractal_atr_band', 'fractal', 'atr_band']
sl_methods = ['fractal']
# choc_pips_diff_thresholds = [5, 10, 15, 20, 25, 30]
choc_pips_diff_thresholds = [1.5, 2.0, 2.5, 3.0]
use_squeeze_vals = [True, False]
use_trailing_sl_vals = [True, False]
risk_reward_ratios = [1.0, 1.5, 2.0]

all_combos = []

for spread_cutoff in spread_cutoffs:
    for sl_method in sl_methods:
        for choc_pips_diff_threshold in choc_pips_diff_thresholds:
            for use_squeeze in use_squeeze_vals:
                for use_trailing_sl in use_trailing_sl_vals:
                    for risk_reward_ratio in risk_reward_ratios:
                        rr_ratio = 0.0 if use_trailing_sl else risk_reward_ratio
                        all_combos.append((spread_cutoff, sl_method, choc_pips_diff_threshold, use_squeeze, use_trailing_sl, rr_ratio))

                        if use_trailing_sl:
                            break

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_spread_cutoff = None
best_sl_method = None
best_choc_pips_diff_threshold = None
best_use_squeeze = None
best_use_trailing_sl = None
best_risk_reward_ratio = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for spread_cutoff, sl_method, choc_pips_diff_threshold, use_squeeze, use_trailing_sl, risk_reward_ratio in combos_to_try:
    reward, day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked,  win_amounts, loss_amounts, trade_dates = run_simulation(spread_cutoff, sl_method, choc_pips_diff_threshold, use_squeeze, use_trailing_sl, risk_reward_ratio)
    runs_finished += 1

    print(reward, day_fees, reward + day_fees)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print(min(win_amounts))
        print(max(win_amounts))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print(min(loss_amounts))
        print(max(loss_amounts))
    print(trade_dates)

    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'spread_cutoff': spread_cutoff, 'sl_method': sl_method, 'choc_pips_diff_threshold': choc_pips_diff_threshold, 'use_squeeze': use_squeeze, 'use_trailing_sl': use_trailing_sl, 'risk_reward_ratio': risk_reward_ratio})


    total_profit = reward + day_fees
    if total_profit > best_reward:
        best_reward = total_profit
        best_spread_cutoff = spread_cutoff
        best_sl_method = sl_method
        best_choc_pips_diff_threshold = choc_pips_diff_threshold
        best_use_squeeze = use_squeeze
        best_use_trailing_sl = use_trailing_sl
        best_risk_reward_ratio = risk_reward_ratio


    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 64

-175.25472999999653 -4.915360000000001 -180.17008999999652
Num buys: 10
Num sells: 16
Num trades: 26
Num wins: 9
Num losses: 17
Win streak: 2
Loss streak: 4
Avg pips risked: 0.004153846153846145
Avg win amount: 74.96801777777782
74.88813999999965
75.08633999999634
Avg loss amount: -49.998052352940995
-49.99983999999954
-49.993590000000154
[Timestamp('2022-06-24 14:05:00'), Timestamp('2022-07-27 14:10:00'), Timestamp('2022-08-31 14:45:00'), Timestamp('2022-09-02 16:40:00'), Timestamp('2022-09-14 04:35:00'), Timestamp('2022-09-15 08:40:00'), Timestamp('2022-09-19 14:50:00'), Timestamp('2022-10-20 04:20:00'), Timestamp('2022-10-21 07:25:00'), Timestamp('2022-10-24 04:55:00'), Timestamp('2022-10-24 13:50:00'), Timestamp('2022-11-02 18:55:00'), Timestamp('2022-11-23 09:05:00'), Timestamp('2022-11-28 08:10:00'), Timestamp('2022-11-30 19:55:00'), Timestamp('2022-12-19 13:15:00'), Timestamp('2022-12-21 14:45:00'), Timestamp('2022-12-23 14:40:00'), Timestamp('2022-12-28 12:55:00')

In [15]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best spread cutoff: ' + str(best_spread_cutoff))
print('Best sl method: ' + str(best_sl_method))
print('Best choc pips difference threshold: ' + str(best_choc_pips_diff_threshold))
print('Best use squeeze val: ' + str(best_use_squeeze))
print('Best use trailing stop loss val: ' + str(best_use_trailing_sl))
print('Best risk reward ratio: ' + str(best_risk_reward_ratio))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 395.928925000012
Best spread cutoff: 0.2
Best sl method: fractal
Best choc pips difference threshold: 30
Best use squeeze val: True
Best use trailing stop loss val: True
Best risk reward ratio: 0.0
-----------------------
Top results:
{'reward': 399, 'spread_cutoff': 0.2, 'sl_method': 'fractal', 'choc_pips_diff_threshold': 30, 'use_squeeze': False, 'use_trailing_sl': False, 'risk_reward_ratio': 3.0}
{'reward': 349, 'spread_cutoff': 0.1, 'sl_method': 'atr_band', 'choc_pips_diff_threshold': 30, 'use_squeeze': False, 'use_trailing_sl': True, 'risk_reward_ratio': 0.0}
{'reward': 250, 'spread_cutoff': 0.2, 'sl_method': 'fractal', 'choc_pips_diff_threshold': 30, 'use_squeeze': True, 'use_trailing_sl': False, 'risk_reward_ratio': 2.0}
{'reward': 199, 'spread_cutoff': 0.2, 'sl_method': 'fractal', 'choc_pips_diff_threshold': 15, 'use_squeeze': True, 'use_trailing_sl': False, 'risk_reward_ratio': 2.0}
{'reward': 250, 'spread_cutoff': 0.2, 'sl_

In [16]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best spread cutoff: ' + str(best_spread_cutoff))
print('Best sl method: ' + str(best_sl_method))
print('Best choc pips difference threshold: ' + str(best_choc_pips_diff_threshold))
print('Best use squeeze val: ' + str(best_use_squeeze))
print('Best use trailing stop loss val: ' + str(best_use_trailing_sl))
print('Best risk reward ratio: ' + str(best_risk_reward_ratio))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 249.99809000002114
Best spread cutoff: 0.2
Best sl method: fractal
Best choc pips difference threshold: 2.5
Best use squeeze val: True
Best use trailing stop loss val: False
Best risk reward ratio: 2.0
-----------------------
Top results:
{'reward': 100, 'spread_cutoff': 0.1, 'sl_method': 'fractal', 'choc_pips_diff_threshold': 3.0, 'use_squeeze': True, 'use_trailing_sl': False, 'risk_reward_ratio': 2.0}
{'reward': 150, 'spread_cutoff': 0.2, 'sl_method': 'fractal', 'choc_pips_diff_threshold': 2.5, 'use_squeeze': True, 'use_trailing_sl': True, 'risk_reward_ratio': 0.0}
{'reward': 99, 'spread_cutoff': 0.2, 'sl_method': 'fractal', 'choc_pips_diff_threshold': 2.0, 'use_squeeze': True, 'use_trailing_sl': False, 'risk_reward_ratio': 2.0}
{'reward': 249, 'spread_cutoff': 0.2, 'sl_method': 'fractal', 'choc_pips_diff_threshold': 2.5, 'use_squeeze': True, 'use_trailing_sl': False, 'risk_reward_ratio': 2.0}
{'reward': 149, 'spread_cutoff': 0.2, 